# Importando bibliotecas

In [21]:
# biblotecas para manipulação de dados
import pandas as pd
import numpy as np

#Bibliotecas auxiliares
import os
from functools import reduce
import re

#Biblioteca propria
import sys
sys.path.append("../src/")
from eda.eda import describe
from io_pyarrow.io_pyarrow import pa_read_csv,pa_write_csv #leitura e escrita de arquivos csv grandes

%reload_ext watermark
%watermark --iversions

pyarrow   : 4.0.0
pandas    : 1.2.2
plotnine  : 0.7.1
seaborn   : 0.11.1
numpy     : 1.20.1
matplotlib: 3.3.3
re        : 2.2.1
sys       : 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]



___

# Importando arquivos

In [4]:
#Definindo path dos arquivos
path_data = "../data/raw/"

In [5]:
def pa_read_csv(source,sep=","):
    return csv.read_csv(source,parse_options=csv.ParseOptions(delimiter=sep)).to_pandas()

## Dados da FAPESP
<b> Descrição </b>: Pasta de arquivos, com dados anonimizados de pacientes que fizeram teste para COVID-19 a partir de 1/11/2019 e respectivos resultados de exames laboratoriais, contendo dentre outros o identificador anonimizado do paciente  <br>
fonte: https://repositoriodatasharingfapesp.uspdigital.usp.br/

### Hospital Albert Einstein

In [6]:
einstein_exames = pa_read_csv(path_data + "FAPESP/EINSTEIN_Exames_2.csv",sep="|")
einstein_pacientes = pa_read_csv(path_data + "FAPESP/EINSTEIN_Pacientes_2.csv",sep="|")

### Grupo Fleury

In [9]:
fleury_exames = pa_read_csv(path_data + "FAPESP/GrupoFleury_Exames_3.csv",sep="|")
fleury_pacientes = pa_read_csv(path_data + "FAPESP/GrupoFleury_Pacientes_3.csv",sep="|")

### Hospital Sírio Libanes

In [10]:
sl_exames = pa_read_csv(path_data + "FAPESP/HSL_Exames_3.csv",sep="|")
sl_pacientes = pa_read_csv(path_data + "FAPESP/HSL_Pacientes_3.csv",sep="|")
sl_desfecho = pa_read_csv(path_data + "FAPESP/HSL_Desfechos_3.csv",sep="|")

### Hospital das Clinicas da Faculdade de Medicina da Universidade de Sao Paulo 

In [11]:
hc_exames = pa_read_csv(path_data + "FAPESP/HC_EXAMES_1.csv",sep="|")
hc_pacientes = pa_read_csv(path_data + "FAPESP/HC_PACIENTES_1.csv",sep="|")

### Beneficência Portuguesa de São Paulo

In [12]:
bpsp_exames = pa_read_csv(path_data + "FAPESP/bpsp_exames_01.csv",sep="|")
bpsp_pacientes = pa_read_csv(path_data + "FAPESP/bpsp_pacientes_01.csv",sep="|")
bpsp_desfecho = pa_read_csv(path_data + "FAPESP/bpsp_desfecho_01.csv",sep="|")

## Portal da Transparência

<b>Descrição</b>: A atualização permanente do número de registros de óbitos em meio a pandemia da Covid-19 é uma iniciativa dos Cartórios de Registro Civil do Brasil <br>
fonte: https://transparencia.registrocivil.org.br/especial-covid

In [15]:
obitos = reduce(lambda x,y: pd.concat((x,y),axis = 0),
                [pa_read_csv(path_data + "Portal_Transparencia/" + file) for file in os.listdir(path_data + "Portal_Transparencia")])

## Fiocruz
fonte: https://bigdata.icict.fiocruz.br/covid-19

### COVID SAUDE
<b>Descrição</b>: Os dados dos estados e municípios são obtidos na página oficial do Ministério da Saúde.<br>
fonte: https://covid.saude.gov.br/

In [16]:
obitos_brasil = pa_read_csv(path_data+"Fiocruz/COVID_MS/HIST_PAINEL_COVIDBR_06mai2021.csv",sep = ";")

## SEADE
fonte: https://www.seade.gov.br/coronavirus/

In [17]:
casos_obitos_doencas_preexistentes = pa_read_csv(path_data+"SEADE/casos_obitos_doencas_preexistentes.csv",sep=";")
casos_obitos_raca_cor = pa_read_csv(path_data+"SEADE/casos_obitos_raca_cor.csv",sep=";")
dados_covid_sp = pa_read_csv(path_data+"SEADE/dados_covid_sp.csv",sep=";")
plano_sp_leitos_internacoes = pa_read_csv(path_data+"SEADE/plano_sp_leitos_internacoes_serie_nova.csv",sep=";")

## OpenData SUS
fonte: https://opendatasus.saude.gov.br/dataset

In [18]:
ocupacao_leitos = pa_read_csv(path_data + "OpenDataSUS/esus-vepi.LeitoOcupacao.csv",sep=";")

___

# Analisando as bases de dados

In [19]:
#Função para uma descrição geral dos dados
def describe(df):
    print("Quantidade de linhas:",df.shape[0])
    df_out = pd.DataFrame(columns = ['variable','type','na' ,'na_pct' ,'unique','min', 'quat25','median','mean', \
                                     'quat75','max','std','skewness','kurtosis','media_desvio'])
    df_out['variable'] = df.columns
    df_out['type'] = df.dtypes.values
    df_out['na'] = [sum(df[coluna].isna()) for coluna in df.columns]
    df_out['na_pct'] = [str(round(100*sum(df[coluna].isna())/df.shape[0],1))+'%' for coluna in df.columns]
    df_out['unique'] = [len(df[coluna].unique()) for coluna in df.columns]
    df_out['min']  = [round(min(df[coluna]),2) if 'int' in str(df[coluna].dtype) or 'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['mean'] = [round(df[coluna].mean(),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['max']  = [round(max(df[coluna]),2) if 'int' in str(df[coluna].dtype) or 'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['std'] = [round(df[coluna].std(),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['quat25'] = [round(df[coluna].quantile(0.25),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['quat75'] = [round(df[coluna].quantile(0.75),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['median'] = [round(df[coluna].quantile(0.5),2) if 'int' in str(df[coluna].dtype) or \
                      'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['skewness'] = [round(df[coluna].skew(),2) if 'int' in str(df[coluna].dtype) or \
                          'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    df_out['kurtosis'] = [round(df[coluna].kurt(),2) if 'int' in str(df[coluna].dtype) or \
                          'float' in str(df[coluna].dtype) else '-' for coluna in df.columns]
    
    df_out_media_desvio_list = []
    for coluna in df.columns:
        if(('int' in str(df[coluna].dtype)) or ('float' in str(df[coluna].dtype)) ):
            if((all(df[coluna] == 0)) or (df[coluna].std() == 0)):
                df_out_media_desvio_list.append(0)
            else:
                df_out_media_desvio_list.append(round(df[coluna].mean()/df[coluna].std(),2))
        else:
            df_out_media_desvio_list.append('-')
    
    df_out['media_desvio'] = df_out_media_desvio_list
    return(df_out)

## FAPESP

### Ajustando dados do Einstein

In [20]:
#Unindos as tabelas de pacientes e exames pelo ID dos pacientes e depois removendo as tabelas intermediárias
einstein = einstein_exames.merge(einstein_pacientes,
                                 how = "left",
                                 on = "ID_PACIENTE")
del(einstein_pacientes)
del(einstein_exames)
describe(einstein)

Quantidade de linhas: 3415155


,variable,type,na,na_pct,unique,min,quat25,median,mean,quat75,max,std,skewness,kurtosis,media_desvio
0,ID_PACIENTE,object,0,0.0%,79840,-,-,-,-,-,-,-,-,-,-
1,DT_COLETA,object,0,0.0%,285,-,-,-,-,-,-,-,-,-,-
2,DE_ORIGEM,object,0,0.0%,1,-,-,-,-,-,-,-,-,-,-
3,DE_EXAME,object,0,0.0%,63,-,-,-,-,-,-,-,-,-,-
4,DE_ANALITO,object,0,0.0%,126,-,-,-,-,-,-,-,-,-,-
5,DE_RESULTADO,object,0,0.0%,27631,-,-,-,-,-,-,-,-,-,-
6,CD_UNIDADE,object,0,0.0%,19,-,-,-,-,-,-,-,-,-,-
7,DE_VALOR_REFERENCIA,object,0,0.0%,308,-,-,-,-,-,-,-,-,-,-
8,IC_SEXO,object,0,0.0%,2,-,-,-,-,-,-,-,-,-,-
9,AA_NASCIMENTO,object,0,0.0%,91,-,-,-,-,-,-,-,-,-,-


#### DT_COLETA

In [31]:
# Algumas informações passam a ser desnecessárias agora
# O ID do paciente somente nos ajuda a fazer o merge das tabelas, mas nao agrega informação
# Algumas colunas, como DE_ORIGEM, CD_PAIS, somente possuem um unico valor, então são desnecessárias
einstein_ = einstein.drop(columns = ["DE_ORIGEM","CD_PAIS"])

#Coluna DT_COLETA é uma variável de data
einstein_.DT_COLETA = pd.to_datetime(einstein.DT_COLETA,errors='coerce')

#### DE_EXAME

In [92]:
#Aparentemente não há problemas com os dados de DE_EXAME
einstein_.DE_EXAME.value_counts().index.to_numpy().reshape(-1,1)

array([['Hemograma com Plaquetas'],
       ['Hemograma Contagem Auto'],
       ['Diferencial Manual'],
       ['Urina Tipo I'],
       ['Gasometria Venosa'],
       ['PCR em tempo real para detecção de Coron'],
       ['Gasometria Arterial'],
       ['Painel Molecular Patogenos Respiratorios'],
       ['Dosagem de Bilirrubina Total e Frações'],
       ['Dosagem de Glicose'],
       ['Dosagem de Uréia'],
       ['Dosagem de CreatiNINA'],
       ['Dosagem de Potássio'],
       ['Dosagem de Sódio'],
       ['Sorologia SARS-CoV-2/COVID19 IgG/IgM'],
       ['Dosagem de TGP'],
       ['TTPA'],
       ['Dosagem de TGO'],
       ['Tempo de Protrombina'],
       ['Dosagem de Gama GT'],
       ['Dosagem de Cálcio Iônico'],
       ['Dosagem de Magnésio'],
       ['Dosagem de Fosfatase Alcalina'],
       ['Dosagem de D-Dímero'],
       ['Dosagem de Ferritina'],
       ['Dosagem de Vitamina B12'],
       ['Dosagem de Fósforo'],
       ['Dosagem de Desidrogenase Láctica'],
       ['Dosagem de Lipase

#### AA_NASCIMENTO

De acordo como  dicionario de dados, AA_NASCIMENTO:

<i>"Os 4 dígitos do ano do nascimento: </i>
* <i> AAAA - para ano de nascimento igual ou anterior a 1930 (visando anonimização) </i>
* <i> YYYY - quaisquer outros anos, em caso de anonimização do ano"</i>

In [32]:
# É possivel notar que existe uma quantidade considerável de anos abaixo de 1930. 
# Vamos converter para numérico, mas tenhamos em mente que os valores abaixo de 1930 serão os faltantes
einstein_.AA_NASCIMENTO.value_counts()

AAAA    186901
1980     64355
1982     63864
1975     61682
1978     61498
         ...  
2015      7815
2006      7786
2013      7109
2005      6794
2011      6790
Name: AA_NASCIMENTO, Length: 91, dtype: int64

In [33]:
einstein_.AA_NASCIMENTO = pd.to_numeric(einstein_.AA_NASCIMENTO,errors='coerce')

#### DE_VALOR_REFERENCIA

In [34]:
#Coluna valor de referencia é uma variável categórica proveninente de uma numérica. Porém, existem alguns valores que estão em branco ou
# que a informação já diz que está ausente (dados faltantes). Dessa forma, vamos considerá-las ausente para fins de análise 
einstein_.DE_VALOR_REFERENCIA.value_counts()

                 752292
Não detectado    197758
Ausentes         109214
6.5 a 15.0        79659
31.0 a 36.0       77335
                  ...  
3,5 a 5,2             1
3.9 a 6.9             1
25,6 a 35,5           1
21 a 53               1
0 a 850               1
Name: DE_VALOR_REFERENCIA, Length: 308, dtype: int64

In [35]:
einstein_.DE_VALOR_REFERENCIA = einstein_.DE_VALOR_REFERENCIA.apply(lambda x: np.nan if (x == "Ausentes") | (x == "") else x)

#### DE_ANALITO

In [36]:
# alguns analitos possuem um sustenido, mas a raiz do analito aparece. Vamos investigar o que pode ser
einstein_.DE_ANALITO.value_counts().index.to_numpy().reshape(-1,1)

array([['Resultado COVID-19:'],
       ['Hematócrito'],
       ['Hemoglobina'],
       ['Plaquetas'],
       ['Hemácias'],
       ['Dosagem de Glicose'],
       ['Volume Médio Plaquetário'],
       ['Leucócitos'],
       ['Leucócitos #'],
       ['VCM'],
       ['CHCM'],
       ['HCM'],
       ['Linfócitos #'],
       ['Monócitos #'],
       ['Linfócitos'],
       ['Monócitos'],
       ['Eosinófilos  #'],
       ['Eosinófilos'],
       ['RDW'],
       ['Basófilos #'],
       ['Basófilos'],
       ['Uréia'],
       ['Neutrófilos'],
       ['Neutrófilos  #'],
       ['Creatinina'],
       ['Potássio'],
       ['Sódio'],
       ['TGP'],
       ['TGO'],
       ['Gama GT'],
       ['Cálcio Iônico mmol/L'],
       ['Magnésio'],
       ['Fosfatase Alcalina'],
       ['Bilirrubina Indireta'],
       ['Bilirrubina Total'],
       ['Bilirrubina Direta'],
       ['Densidade'],
       ['pH'],
       ['Esterase'],
       ['Proteína'],
       ['Açúcar'],
       ['Hemoglobina Urina'],
       ['Aspect

In [39]:
#Ao que parece, o analito com # possui uma unidade de medida diferente
einstein_[einstein_["DE_ANALITO"].isin(['Mieloblastos','Mieloblastos #'])].head()

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA,IC_SEXO,AA_NASCIMENTO,CD_UF,CD_MUNICIPIO,CD_CEPREDUZIDO
503,000bb5ffdfd9bfd09e40294697e95a196f299307,2020-02-06,Diferencial Manual,Mieloblastos,0.0,%,NaN,M,1944.0,SP,SAO PAULO,CCCC
515,000bb5ffdfd9bfd09e40294697e95a196f299307,2020-02-06,Diferencial Manual,Mieloblastos #,0,µL,0 a 0,M,1944.0,SP,SAO PAULO,CCCC
699,000bb5ffdfd9bfd09e40294697e95a196f299307,2020-06-05,Diferencial Manual,Mieloblastos #,0,µL,0 a 0,M,1944.0,SP,SAO PAULO,CCCC
732,000bb5ffdfd9bfd09e40294697e95a196f299307,2020-06-05,Diferencial Manual,Mieloblastos,0.0,%,NaN,M,1944.0,SP,SAO PAULO,CCCC
934,000bb5ffdfd9bfd09e40294697e95a196f299307,2020-10-08,Diferencial Manual,Mieloblastos #,0,µL,0 a 0,M,1944.0,SP,SAO PAULO,CCCC


In [50]:
#Vamos confirmar se as variaveis que possuem # possuem unidades de medida diferentes
var_sustenido = einstein_[einstein_["DE_ANALITO"].str.contains(" #")].DE_ANALITO.unique()
var_raiz = np.array([[var.split(" ")[0],var] for var in var_sustenido]).flatten()

einstein_[einstein_["DE_ANALITO"].apply(lambda x: x in var_raiz)].groupby(["DE_ANALITO","CD_UNIDADE"]).ID_PACIENTE.size().sort_index()

DE_ANALITO        CD_UNIDADE
Bastonetes #      µL            17679
Basófilos         %             78554
Basófilos #       µL            78656
Eosinófilos       %             78777
Eosinófilos  #    µL            78813
Leucócitos        x10^3/uL      79152
Leucócitos #      µL            79152
Linfócitos        %             79011
Linfócitos #      µL            79034
Metamielócitos    %             17538
Metamielócitos #  NULL              1
                  µL            17592
Mieloblastos      %             17299
Mieloblastos #    NULL              2
                  µL            17295
Mielócitos        %             17460
Mielócitos #      NULL              1
                  µL            17491
Monócitos         %             78998
Monócitos #       µL            79028
Neutrófilos       %             61338
Neutrófilos  #    µL            61332
Promielócitos     %             17316
Promielócitos #   NULL              2
                  µL            17317
Segmentados       %  

Além de comprovar que o # representa uma unidade de medida diferente, ainda foi possível encontrar quais deveriam ser os valores a serem preenchidos no lugar de NULL

In [82]:
#Primeiro, vamos atribuir NaN aos valores NULL
einstein_["CD_UNIDADE"] = einstein_["CD_UNIDADE"].replace({"NULL":np.nan})

In [83]:
# Em seguida, recuperar quem sao os valores a serem mapeados e realizar o mapeamento
analito_unidade = einstein_[einstein_["DE_ANALITO"].apply(lambda x: x in var_raiz)].groupby(["DE_ANALITO","CD_UNIDADE"]).ID_PACIENTE.size().sort_values(ascending = False)
einstein_["CD_UNIDADE"] = einstein_["CD_UNIDADE"].fillna(einstein_["DE_ANALITO"].map(analito_unidade.reset_index().groupby("DE_ANALITO").CD_UNIDADE.first().to_dict()))

In [84]:
#Mapeamento corrigido
einstein_[einstein_["DE_ANALITO"].apply(lambda x: x in var_raiz)].groupby(["DE_ANALITO","CD_UNIDADE"]).ID_PACIENTE.size().sort_index()

DE_ANALITO        CD_UNIDADE
Bastonetes #      µL            17679
Basófilos         %             78554
Basófilos #       µL            78656
Eosinófilos       %             78777
Eosinófilos  #    µL            78813
Leucócitos        x10^3/uL      79152
Leucócitos #      µL            79152
Linfócitos        %             79011
Linfócitos #      µL            79034
Metamielócitos    %             17538
Metamielócitos #  µL            17593
Mieloblastos      %             17299
Mieloblastos #    µL            17297
Mielócitos        %             17460
Mielócitos #      µL            17492
Monócitos         %             78998
Monócitos #       µL            79028
Neutrófilos       %             61338
Neutrófilos  #    µL            61332
Promielócitos     %             17316
Promielócitos #   µL            17319
Segmentados       %             17685
Segmentados #     µL            17707
Name: ID_PACIENTE, dtype: int64

#### DE_RESULTADO

De acordo com o dicinário de dados, 

<i>"Se DE_ANALITO exige valor numérico, Inteiro ou Decimal, se DE_ANALITO exige qualitativo, String com domínio restrito"</i>

Portanto, há valores numericos e não numericos. Vamos dividir em duas colunas para facilitar nossa analise. Mas antes, vamos ver se não haverá nenhum problema na conversão

In [101]:
teste_sanidade_DE_RESULTADO = pd.DataFrame({"Conversao":pd.to_numeric(einstein_.DE_RESULTADO,errors='coerce'),
                                            "HaNumero":einstein_.DE_RESULTADO.apply(lambda x: len(re.findall("[0-9]+",x)) > 0)})
teste_sanidade_DE_RESULTADO

,Conversao,HaNumero
0,863.00,True
1,13.00,True
2,134.00,True
3,1.30,True
4,24.00,True
...,...,...
3415150,41.40,True
3415151,NaN,False
3415152,0.26,True
3415153,0.55,True


In [102]:
# Vamos verificar se existe algum caso que dizia que havia numero mas nao houve conversão
teste_sanidade_DE_RESULTADO.query("HaNumero == True & Conversao != Conversao")

,Conversao,HaNumero
161,NaN,True
286,NaN,True
320,NaN,True
332,NaN,True
610,NaN,True
...,...,...
3415039,NaN,True
3415051,NaN,True
3415054,NaN,True
3415060,NaN,True


In [107]:
# Nota-se que existem casos que existem numeros, mas a conversão não foi possível. Vamos verificar o motivo no original
einstein_.iloc[teste_sanidade_DE_RESULTADO.query("HaNumero == True & Conversao != Conversao").index,:]["DE_RESULTADO"].value_counts()

<1000                7256
1+ (~30 mg/dL)       1983
1+ (~25 WBCs/µL)     1674
<215                 1546
3+ (~500 WBCs/µL)     964
                     ... 
118,3                   1
158,9                   1
2,08                    1
85,6                    1
1,00                    1
Name: DE_RESULTADO, Length: 456, dtype: int64

In [127]:
# Em alguns casos, temos números mesmo, mas por conta da "," não foi possível a conversão correta. Vejamos desses casos, aqueles que possuem "," para o devido tratamento
de_resultado_verificar_numero = einstein_.iloc[teste_sanidade_DE_RESULTADO.query("HaNumero == True & Conversao != Conversao").index,:]["DE_RESULTADO"]
index_converter_virgula = pd.to_numeric(de_resultado_verificar_numero[de_resultado_verificar_numero.apply(lambda x: "," in x)].str.replace(",","."),errors="coerce").isna()
index_converter_virgula.value_counts()

False    680
True      16
Name: DE_RESULTADO, dtype: int64

In [138]:
# Ainda há casos com virgula que não é possivel converter. Vamos ver do que se trata
de_resultado_verificar_numero[index_converter_virgula[index_converter_virgula==True].index]

243343        2,5 L O2/min
575482           0,5 LITRO
1038068       3,0 L O2/min
1113748              21,0%
1624244       3,0 L O2/min
1893670       2,0 L O2/min
1963099       2,0 L O2/min
2046949             >160,0
2572943    7,0 L de O2/min
2646327             >160,0
2685704       2,0 L O2/min
2775121             >160,0
2946838         0,5 Litros
2949733             >160,0
2974611          ?32.737,0
3402611             >160,0
Name: DE_RESULTADO, dtype: object

Como podemos ver, podemos fazer o seguinte tratamento, para resolver a questão:
* ",0" pode ser substituido por "" sem perda de informação
* ? pode representar um erro. Vamos verificar

In [159]:
# Ao que tudo, indica, trata-se de erro de digitação. Vamos confirmar
einstein_[einstein_.DE_RESULTADO.str.contains('\?')]

,ID_PACIENTE,DT_COLETA,DE_EXAME,DE_ANALITO,DE_RESULTADO,CD_UNIDADE,DE_VALOR_REFERENCIA,IC_SEXO,AA_NASCIMENTO,CD_UF,CD_MUNICIPIO,CD_CEPREDUZIDO
182107,0d84bbdd3063f9f599b3b2aa54eb8ecf0f045983,2020-05-15,Gasometria Arterial,Ácido Lático Arterial,?1,mg/dL,4.5 a 14.4,M,1939.0,SP,SAO PAULO,CCCC
182110,0d84bbdd3063f9f599b3b2aa54eb8ecf0f045983,2020-05-15,Dosagem de Lactato (arterial),Ácido Lático Arterial,?1,mg/dL,4.5 a 14.4,M,1939.0,SP,SAO PAULO,CCCC
774376,3ac6af7bd5c04a8534c751454dfc0cbc720bef6d,2020-04-29,Gasometria Venosa,Saturação da Hb (gasometria venosa),?.....,%,40.0 a 70.0,M,1971.0,SP,SAO PAULO,CCCC
2105506,a37dd7ecccd88383831b95621ca8cae35deb1190,2020-04-26,Gasometria Arterial,ctO2 (gasometria arterial),?3.0,mmol/L,8.4 a 9.9,M,1958.0,SP,SAO PAULO,CCCC
2974611,e2220446ffaa6b84e5877a05333e27d142a505c9,2019-12-23,Gasometria Venosa,pO2 (gasometria venosa),"?32.737,0",mm Hg,25.0 a 40.0,M,1935.0,SP,SAO PAULO,CCCC


In [160]:
#Exames com "Gasometria" possuem valores numericos sem "?", então podemos remover sem perda de informação
einstein_[einstein_.DE_EXAME.str.contains('Gasometria')].DE_RESULTADO.value_counts()

21      9092
2       2056
30      1392
40      1353
        1064
        ... 
0,7        1
27,4       1
59,4       1
63         1
19,0       1
Name: DE_RESULTADO, Length: 2407, dtype: int64

In [172]:
#Portando agora, podemos fazer a conversão considerando o teste de sanidade HaNumero == True & Conversao == NaN pois está verificado
de_resultado_aux = einstein_.DE_RESULTADO.str.replace(",0","").str.replace("\?","")

C:\Users\ohbto\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.


In [190]:
#Vamos seperar os dados em resultados numericos e resultados nao numericos
einstein_["DE_RESULTADO_NUMERICO"] = pd.to_numeric(de_resultado_aux,errors='coerce')
einstein_["DE_RESULTADO_NAO_NUMERICO"] = einstein_.apply(lambda x: x["DE_RESULTADO"] if pd.isna(x["DE_RESULTADO_NUMERICO"]) else np.nan,axis = 1)

#### CD_UNIDADE

In [204]:
#Não há uma explicação clara da falta de valores
einstein_.groupby(["DE_RESULTADO","CD_UNIDADE"],dropna=False).ID_PACIENTE.size().reset_index().query("CD_UNIDADE != CD_UNIDADE")

,DE_RESULTADO,CD_UNIDADE,ID_PACIENTE
0,,NaN,49603
17,*,NaN,252
22,**,NaN,50
24,***,NaN,17
27,*NOVA COLETA,NaN,2
...,...,...,...
44587,nova coleta,NaN,268
44589,vide nota,NaN,2
44591,Ácido Úrico +++,NaN,20
44592,Ácido Úrico -++,NaN,2


#### CD_UF

In [209]:
# Há uma UF == UU
einstein_.CD_UF.value_counts()

SP    3256973
UU     155273
MG       2896
RJ         13
Name: CD_UF, dtype: int64

In [213]:
# Esses casos ocorrem quando, de acordo com o dicionario de dados, há necessida de anonimização ou o paciente é estrangeiro
einstein_[einstein_.CD_UF == "UU"].CD_MUNICIPIO.value_counts()

MMMM    155273
Name: CD_MUNICIPIO, dtype: int64

In [217]:
einstein_.CD_UF = einstein_.CD_UF.replace({"UU":np.nan})

#### CD_MUNICIPIO

In [218]:
einstein_.CD_MUNICIPIO.value_counts()

SAO PAULO                2550567
MMMM                      526886
SANTANA DE PARNAIBA       116978
BARUERI                    98919
OSASCO                     19321
COTIA                      18130
JUNDIAI                    14165
GUARULHOS                  10959
SAO BERNARDO DO CAMPO       9289
TABOAO DA SERRA             8405
CARAPICUIBA                 8026
SANTO ANDRE                 7542
SANTOS                      4560
CAMPINAS                    4234
EMBU                        3274
SOROCABA                    2380
DIADEMA                     2301
JANDIRA                     2231
SAO CAETANO DO SUL          2006
ITAPECERICA DA SERRA        1413
ITAPEVI                     1183
MAUA                         503
SAO JOSE DOS CAMPOS          441
CAJAMAR                      340
ATIBAIA                      312
PIRACICABA                   157
JARINU                       157
CABREUVA                     142
MOGI DAS CRUZES              118
CAIEIRAS                      47
PRAIA GRAN

In [220]:
# Corrigindo os dados anonimizados do municipio
einstein_.CD_MUNICIPIO = einstein_.CD_MUNICIPIO.replace({"MMMM",np.nan})

#### CD_CEPREDUZIDO

In [223]:
#De acordo com o dicionario de dados, os CEP CCC são anonimizados. Porém, mais de 98% dos dados não possuem a informação correta de CEP. Portanto, iremos remover da base
einstein_.CD_CEPREDUZIDO.value_counts(normalize=True)

CCCC        0.986184
05652900    0.008932
05679010    0.000914
13213080    0.000627
06543001    0.000606
              ...   
04738000    0.000001
14960000    0.000001
14802632    0.000001
13026275    0.000001
13486710    0.000001
Name: CD_CEPREDUZIDO, Length: 63, dtype: float64

In [224]:
einstein_ = einstein_.drop(columns = "CD_CEPREDUZIDO")

In [225]:
# Describe final
describe(einstein_)

Quantidade de linhas: 3415155


,variable,type,na,na_pct,unique,min,quat25,median,mean,quat75,max,std,skewness,kurtosis,media_desvio
0,ID_PACIENTE,object,0,0.0%,79840,-,-,-,-,-,-,-,-,-,-
1,DT_COLETA,datetime64[ns],0,0.0%,285,-,-,-,-,-,-,-,-,-,-
2,DE_EXAME,object,0,0.0%,63,-,-,-,-,-,-,-,-,-,-
3,DE_ANALITO,object,0,0.0%,126,-,-,-,-,-,-,-,-,-,-
4,CD_UNIDADE,object,669174,19.6%,19,-,-,-,-,-,-,-,-,-,-
5,DE_VALOR_REFERENCIA,object,861506,25.2%,307,-,-,-,-,-,-,-,-,-,-
6,IC_SEXO,object,0,0.0%,2,-,-,-,-,-,-,-,-,-,-
7,AA_NASCIMENTO,float64,186901,5.5%,91,1931.0,1950.0,1967.0,1967.76,1982.0,2020.0,21.45,0.32,-0.56,91.73
8,CD_UF,object,155273,4.5%,4,-,-,-,-,-,-,-,-,-,-
9,CD_MUNICIPIO,object,0,0.0%,43,-,-,-,-,-,-,-,-,-,-


In [226]:
pa_write_csv(einstein_,"../data/interim/eistein.csv")

In [194]:
# sl = sl_exames.merge(sl_pacientes,
#                      how = 'left',
#                      on = 'ID_PACIENTE').merge(sl_desfecho,
#                                                how = 'left',
#                                                on = ['ID_PACIENTE','ID_ATENDIMENTO'])
# del(sl_exames)
# del(sl_pacientes)
# del(sl_desfecho)
# describe(sl)

In [193]:
# hc = hc_exames.merge(hc_pacientes,
#                      on = 'ID_PACIENTE',
#                      how = 'left')
# del(hc_pacientes)
# del(hc_exames)
# describe(hc)

In [192]:
# bpsp = bpsp_exames.merge(bpsp_pacientes,
#                          on = "ID_PACIENTE",
#                          how = 'left').merge(bpsp_desfecho,
#                                               how = 'left',
#                                               on = ['ID_PACIENTE','ID_ATENDIMENTO'])
# describe(bpsp)